### Using Fortran and f2py

We will replace the `jacobi.py` implementation with an external
Fortran subroutine that performs the same calculation.

If you wish to try to write your own Fortran Jacobi code, ignore
the accompanying `jacobi.f90` and edit new file; you will need to
adjust the commands below.

Otherwise, you should be able to locate `jacobi.f90` and the accompanying
`Makefile` in the same directory that contains this notebook.
Look at the Fortran source provided. The `Makefile` is there
for convenience and contains relevant targets for the Fortran
version. This notebook uses the commands explicitly.

There are a number of things to note about `jacobi.f90`.

The lines with `!f2py` are used to define the attributes of the
argument variables to the `f2py` tool. `intent(in)` specifies that
the variables are input variables only, `intent(inplace)` specifies
that the variable is being passed by reference (so any changes to
the variable in this routine will be reflected in the calling code).

We now need to use the f2py tool to produce a signature of this Fortran subroutine that specifies how to construct the interface between Python and Fortran. You do this with the following shell command:
```
$ f2py jacobi.f90 -m jacobi -h jacobi.pyf
```

In [ ]:
# Or try it here via a shell-escape
! f2py jacobi.f90 -m jacobi -h jacobi.pyf

This should produce a text file called `jacobi.pyf` that contains
the signature for the Fortran subroutine `jacobi()`. We now need
to combine this signature with the actual Fortran code to produce
a dynamic library that can be imported by Python. We use the `f2py`
command again to do this (note that this stage requires a Fortran
compiler to be available):
```
$ f2py -c jacobi.pyf jacobi.f90
```

In [ ]:
# Here via the shell escape (Can produce a lot of output...)
! f2py -c jacobi.pyf jacobi.f90

This should produce (after some output) a file with a `.so` extension
(e.g., on MacOS with python3.7 the file is `jacobi.cpython-37m-darwin.so`).

We can test it works via the following, which should produce the
docstring for the Fortran function:


In [ ]:
import jacobi
print(jacobi.jacobi.__doc__)

As you can see, the calling sequence from Python differs from the
argument sequence specified in the Fortran code. The two arguments
`m` and `n` are not required when calling from Python. f2py has
realized that these variables are related to the dimensions of the
`psi` array. The interface wrapper knows how to extract these values
from the `numpy` array in Python so you can just call the function
via:
```python
from jacobi import jacobi
...
jacobi(niter, psi)
```

You should now be able to verify the code works, and test the performance.

In [ ]:
# E.g., and compare velocity.dat with ../verify/cfd_velocity_1_1000dat
! ./cfd.py 1 1000
! diff velocity.dat ../verify/cfd_velocity_1_1000.dat